In [3]:
import transformers
from langtest import Harness

ModuleNotFoundError: No module named 'langtest'

In [4]:
!pip install langtest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 20.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 42.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-

In [2]:
!nvidia-smi

Tue Jan 23 12:14:54 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   39C    P8     8W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Import and create a Harness object
# from langtest import Harness
h = Harness(task='ner', model={"model":'dslim/bert-base-NER', "hub":'huggingface'})

Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  },
  "fairness": {
   "min_gender_f1_score": {
    "min_score": 0.6
   }
  },
  "representation": {
   "min_label_representation_count": {
    "min_count": 50
   }
  }
 }
}


In [4]:
# # Generate test cases, run them and view a report
h.generate().run().report()

Generating testcases...: 100%|██████████| 5/5 [00:00<00:00, 28416.69it/s]
[W010] - Test 'add_typo': 9 samples removed out of 226
[W010] - Test 'american_to_british': 226 samples removed out of 226

[W010] - Test 'replace_to_female_pronouns': 198 samples removed out of 226
[W010] - Test 'replace_to_low_income_country': 138 samples removed out of 226

Running testcases... : 100%|██████████| 342/342 [01:14<00:00,  4.60it/s]


,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,34,183,84%,70%,True
1,accuracy,min_micro_f1_score,0,1,100%,100%,True
2,bias,replace_to_female_pronouns,1,27,96%,70%,True
3,bias,replace_to_low_income_country,4,84,95%,70%,True
4,fairness,min_gender_f1_score,0,3,100%,100%,True
5,representation,min_label_representation_count,0,5,100%,100%,True


## Mistral

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
model_name = "mistralai/Mistral-7B-v0.1"

In [ ]:
# CPU

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [7]:
# Define the input text
input_text = "How much is 4 + 3"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate the output
output = model.generate(input_ids)

# Decode the output
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)

How much is 4 + 3 + -1 + -1 + -1 + -


In [2]:
# GPU

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Load the model and move it to the device
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   4 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                        │
│   5                                                                                              │
│   6 # Load the model and move it to the device                                                   │
│ ❱ 7 model = AutoModelForCausalLM.from_pretrained(model_name).to(device)                          │
│   8 tokenizer = AutoTokenizer.from_pretrained(model_name)                                        │
│   9                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:2271 in to                 │
│                                                                                                  │
│   2268 │   │   │   │   │   "You cannot cast a GPTQ model in a new `dtype`. Make sure to load th  │
│   2269 │   │   │   │   │   " `dtype` by passing the correct `torch_dtype` argument."             │
│   2270 │   │   │   │   )                                                                         │
│ ❱ 2271 │   │   return super().to(*args, **kwargs)                                                │
│   2272 │                                                                                         │
│   2273 │   def half(self, *args):                                                                │
│   2274 │   │   # Checks if the model is quantized                                                │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1160 in to                     │
│                                                                                                  │
│   1157 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1158 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1159 │   │                                                                                     │
│ ❱ 1160 │   │   return self._apply(convert)                                                       │
│   1161 │                                                                                         │
│   1162 │   def register_full_backward_pre_hook(                                                  │
│   1163 │   │   self,                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:810 in _apply                  │
│                                                                                                  │
│    807 │   def _apply(self, fn, recurse=True):                                                   │
│    808 │   │   if recurse:                                                                       │
│    809 │   │   │   for module in self.children():                                                │
│ ❱  810 │   │   │   │   module._apply(fn)                                                         │
│    811 │   │                                                                                     │
│    812 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    813 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                            

## Mistral 4bit quantized

In [3]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# To make our life easy, we will use the pipeline function from the Transformers library to generate the response based on the prompt.
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

In [8]:
prompt = "As a data scientist, can you explain the concept of regularization in machine learning?"

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

As a data scientist, can you explain the concept of regularization in machine learning?

> Regularization is a technique that helps reduce overfitting and improves generalization in machine learning models. It involves adding a penalty term to the objective function of the model, which encourages simpler models with fewer parameters.

##### 4) What is a feature?

> A feature is a characteristic or attribute of a data point that is used to describe and distinguish between different observations in a dataset. Features can be numerical or categorical, continuous or discrete, and


## Fine-tune Mistral

In [9]:
# update and install the necessary Python libraries
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [7]:
# load the necessary modules for effective fine-tuning of the model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [14]:
!huggingface-cli login --token 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
# monitor LLM performance
wandb.login()
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: Currently logged in as: huertas_97. Use `wandb login --relogin` to force relogin


In [3]:
api = wandb.Api()
username = api.api_key
organization = api.default_entity
print("Organización:", organization)

Organización: huertas_97


In [4]:
# set the base model, dataset, and new model name.
base_model = "mistralai/Mistral-7B-v0.1"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b_guanaco"

In [5]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train")
dataset["text"][100]

'<s>[INST] cuanto es 2x2 xD [/INST] La respuesta es 4. </s><s>[INST] puedes demostrarme matematicamente que 2x2 es 4? [/INST] En una multiplicación, el producto es el resultado de sumar un factor tantas veces como indique el otro, es decir, si tenemos una operación v · n = x, entonces x será igual a v sumado n veces o n sumado v veces, por ejemplo, para la multiplicación 3 · 4 podemos sumar "3 + 3 + 3 + 3" o "4 + 4 + 4" y en ambos casos nos daría como resultado 12, para el caso de 2 · 2 al ser iguales los dos factores el producto sería "2 + 2" que es igual a 4 </s>'

In [6]:
# Load a model using 4-bit precision 
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Load the tokenizer and configure it to fix the issue with fp16
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [8]:
# PEFT - include an adopter layer in our model. This will enable us to fine-tune the model using a small number
# of parameters, making the entire process faster and more memory-efficient

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [9]:
# Hyperparameters

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [10]:
# Supervised fine-tuning (SFT) Trainer with the necessary components, such as the model, dataset, 
# Lora configuration, tokenizer, and training parameters

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)


/usr/local/lib/python3.9/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [11]:
# Train the model

trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.034100
50,1.476700
75,1.073900
100,1.351800
125,1.049600
150,1.248100
175,1.066600
200,1.297200
225,1.004700
250,1.340700


/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.uti

TrainOutput(global_step=250, training_loss=1.194345947265625, metrics={'train_runtime': 3118.4423, 'train_samples_per_second': 0.321, 'train_steps_per_second': 0.08, 'total_flos': 1.874641569231667e+16, 'train_loss': 1.194345947265625, 'epoch': 1.0})

In [ ]:
# 
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

In [ ]:
!huggingface-cli login --token $secret_hf

In [ ]:
trainer.model.push_to_hub("", use_temp_dir=False)

In [13]:
from transformers import AutoModel, AutoModelForCausalLM

# Replace 'path_to_best_checkpoint' with the actual path of the best checkpoint
model = AutoModelForCausalLM.from_pretrained('/notebooks/notebooks/results/checkpoint-225')
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('/notebooks/notebooks/results/checkpoint-225')


In [22]:
model.push_to_hub(new_model)

/usr/local/lib/python3.9/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Huertas97/mistral_7b_guanaco/commit/12df2ce917b617d6a29f516e26b98873046f7bd9', commit_message='Upload model', commit_description='', oid='12df2ce917b617d6a29f516e26b98873046f7bd9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.model.push_to_hub(new_model, use_temp_dir=False)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

In [23]:
import os

os.listdir("/notebooks/notebooks/results/checkpoint-225")

['trainer_state.json',
 'tokenizer.model',
 'README.md',
 'training_args.bin',
 'optimizer.pt',
 'tokenizer.json',
 'adapter_config.json',
 'rng_state.pth',
 'scheduler.pt',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'adapter_model.safetensors']

In [ ]:
# base_model = "mistralai/Mistral-7B-v0.1"

from peft import PeftModel

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)

model = PeftModel.from_pretrained(base_model_reload, "/notebooks/notebooks/results/checkpoint-225")


In [ ]:
from transformers import logging
logging.set_verbosity(logging.CRITICAL)

prompt = "How do I find true love?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# load the necessary modules for effective fine-tuning of the model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

base_model = "mistralai/Mistral-7B-v0.1"
# base_model_reload = AutoModelForCausalLM.from_pretrained(
#         base_model,
#         return_dict=True,
#         low_cpu_mem_usage=True,
#         device_map="cuda",
#         trust_remote_code=True,
# )

# Load a model using 4-bit precision 
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model = PeftModel.from_pretrained(base_model_reload, "/notebooks/notebooks/results/checkpoint-225")

In [2]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [3]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [5]:
prompt = "How become a AI certified data professional"

sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] How become a AI certified data professional [/INST] To become an AI certified data professional, you will need to obtain a relevant certification. There are several certifications available, including the Certified Analytics Professional (CAP) from the Institute for Business Analytics, the Data Science Certificate from the University of California, Berkeley, and the Data Science Certification from the University of California, Irvine. These certifications will require you to demonstrate your knowledge and skills in data science and analytics. Additionally, you may want to consider pursuing a graduate
